In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import sys
from datetime import datetime
from datetime import timedelta
import xlsxwriter

# Data Processing helpers

In [2]:
data_path = r"C:\Users\User\Desktop\Math402w\Math402W Proj\907\morning" 
# put r before your normal string it converts normal string to raw string:

#processes the time matrix
time_matrix = pd.read_csv(data_path+'/time_matrix.csv', index_col = 'Unnamed: 0')
processed_time_matrix = time_matrix.values.tolist()

In [13]:
time_slot_data = pd.read_csv(data_path+'/time_slot.csv')
#pd.read_csv(data_path+'/903_time_slot_AM.csv', index_col = 'Unnamed: 0')
#creates a tuple of time data
processed_time_windows = time_slot_data[["start_time_scaled","end_time_scaled"]].apply(tuple, axis = 1).to_list()

In [44]:
##
time_slot_data_forcsv = pd.read_csv(data_path+'/time_slot.csv', index_col=[0])
time_slot_data_forcsv=time_slot_data_forcsv.reset_index()
time_slot_data_forcsv.rename(columns={'dummy_add':'address'}, inplace=True )
time_slot_data_forcsv=time_slot_data_forcsv.replace('\+',' ',regex=True)
time_slot_data_forcsv.drop(['slots', 'start_time_scaled','end_time_scaled'], axis = 1,inplace = True)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('testformultiple_907.xlsx', engine='xlsxwriter')
writer_droppedNodes = pd.ExcelWriter('dropped_907.xlsx', engine='xlsxwriter')

In [45]:
time_slot_data_forcsv

,index,address,start_time,end_time
0,0,Add_0,6:45:00,14:00:00
1,1,Add_3,9:00:00,11:00:00
2,2,Add_4,12:00:00,14:00:00
3,3,Add_5,7:00:00,14:00:00
4,4,Add_6,9:00:00,11:00:00
5,5,Add_9,7:00:00,14:00:00
6,6,Add_10,9:00:00,11:00:00
7,7,Add_11,9:00:00,11:00:00
8,8,Add_12,7:00:00,9:00:00
9,9,Add_13,12:00:00,14:00:00


In [46]:
processed_time_windows

[(0, 435),
 (135, 255),
 (315, 435),
 (15, 435),
 (135, 255),
 (15, 435),
 (135, 255),
 (135, 255),
 (15, 135),
 (315, 435),
 (15, 435),
 (15, 435),
 (15, 135),
 (15, 435),
 (15, 435),
 (15, 135),
 (15, 135),
 (315, 435),
 (315, 435),
 (15, 135),
 (135, 255)]

# Config

In [47]:
#helper functions
def addServiceTime(time_matrix, service_time):
    for from_idx in range(1,len(time_matrix)):
        for to_idx in range(len(time_matrix[0])):
            if from_idx != to_idx:
                time_matrix[from_idx][to_idx] = service_time + time_matrix[from_idx][to_idx]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = processed_time_matrix
    data['time_windows'] = processed_time_windows
    data['num_vehicles'] = 2
    data['num_locations'] = len(data['time_windows']) - 1
    data['depot'] = 0 #depot location based on item
    
    #driver breaks
    data['driver_break_duration'] = 30
    data['driver_break_start'] = 240 # hours past 7AM * 60 minutes/hour
    data['driver_break_end'] = 360 
    data['breaks'] = [data['driver_break_duration'] for _ in range(data['num_vehicles'])]
    
    #service times
    data['service_time'] = 12
    data['maximum_wait_at_location'] = 10000
    addServiceTime(data['time_matrix'], data['service_time'])
    
    
    #vehicle capacities
    data['demands'] = [0] + [1 for _ in range(data['num_locations'])]
    data['max_vehicle_capacities'] = 21
    load_balancing = False;
    limit_vehicle_capacity = 0.8 #only applied if load_balancing = true
    
    if load_balancing == True:
        data['vehicle_capacities'] = [data['max_vehicle_capacities']*limit_vehhicle_capacity for _ in range(data['num_vehicles'])]
    else:
        data['vehicle_capacities'] = [data['max_vehicle_capacities'] for _ in range(data['num_vehicles'])]
    
    #solver config
    data['solution_limit'] = 40 #40000 #upper bound of # of solutions to calculate
    data['drop_penalty'] = 99999
    
    #set time boundaries

    return data


def print_solution(data, manager, routing, solution, final, dropped_address):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    #prints breaks
    print('Breaks:')
    intervals = solution.IntervalVarContainer()
    for i in range(intervals.Size()):
        brk = intervals.Element(i)
        if brk.PerformedValue() == 1:
            print('{}: Start({}) Duration({})'.format(
                brk.Var().Name(),
                brk.StartValue(),
                brk.DurationValue()))
            ##
            break_value = brk.StartValue()
            
        else:
            print('{}: Unperformed'.format(brk.Var().Name()))
    print()
    
    
    #Prints dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
            a = time_slot_data_forcsv[time_slot_data_forcsv['index']==node]
            dropped_address = dropped_address.append(a,ignore_index= True)
            
    #print(dropped_nodes, '\n')
    
    
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    
    ##
    current_time_str = '07/03/2022 06:45'
    date_format_str = '%d/%m/%Y %H:%M'
    # create datetime object from timestamp string
    current_time = datetime.strptime(current_time_str, date_format_str)
    i = 1  # use to change time 
    sheet_num=1 #use to name the tabs in excel 
    break_start_time = current_time + pd.DateOffset(minutes=break_value)
    break_end_time = break_start_time + pd.DateOffset(minutes=30)
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        
        isFirstNode = True
        #print(routing.End(index))
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            
            if isFirstNode:          
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var))
                
                a = time_slot_data_forcsv[time_slot_data_forcsv['index']==index]
                final=final.append(a,ignore_index= True)
                n=solution.Max(time_var)
                final['start_time'] = current_time + timedelta(minutes = n) #leave time of the first order
                final['end_time'] = final['start_time']     # the leave time from the store
                
                index = solution.Value(routing.NextVar(index))
                isFirstNode = False
                #
                n=solution.Min(time_var)
                a = time_slot_data_forcsv[time_slot_data_forcsv['index']==index]
                final=final.append(a,ignore_index= True)
                final['start_time'].loc[i] = current_time + timedelta(minutes = n) 
                start_time = final['start_time']
                s = data['service_time']
                end_time = start_time  + timedelta(minutes=s)
                #end_time = final['start_time']  + pd.DateOffset(minutes=s)
                final['end_time'].loc[i] = end_time
                check_value_last_end = solution.Min(time_var) # add here so we can check at the first iteration
                print(final)
                
            else:
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var) + data["service_time"])
                index = solution.Value(routing.NextVar(index))
                 ##
                check_value = solution.Min(time_var)
                #print(check_value)
                if (check_value_last_end <= break_value) & (check_value > break_value): # if break happens after the last order and before next 
                    final.loc[i-1] = ["break_time", "break_time", break_start_time, break_end_time ]
                    n=solution.Min(time_var) + 30
                else:
                    n=solution.Min(time_var)
                    
                a = time_slot_data_forcsv[time_slot_data_forcsv['index']==index]
                final=final.append(a,ignore_index= True)
                final['start_time'].loc[i] = current_time + timedelta(minutes = n)
                #print(final.loc[:,2])
                start_time = final['start_time'].values[i]
                s = data['service_time']
                #end_time = start_time  + pd.DateOffset(minutes=s)
                end_time = start_time  + timedelta(minutes=s)
                final['end_time'].loc[i] = end_time
                check_value_last_end = solution.Min(time_var) 
                i+=1
            
            
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))    
       
        print(plan_output)
        print(final)
        print(break_value)
        total_time += solution.Min(time_var)
        ## import multiple dataframes into different worksheets
        final.to_excel(writer, sheet_name="Sheet" + str(sheet_num),index=False)
        dropped_address.to_excel(writer_droppedNodes, sheet_name="Sheet" + str(sheet_num),index=False)
        sheet_num+=1
        
    writer.save()    
    writer_droppedNodes.save()
    #final.to_csv('test1.csv', index=False)
    dropped_address.to_csv('dropped_address.csv', index=False)
    
    print('Total time of all routes: {}min'.format(total_time))
    




# TSP Solver

In [48]:
"""Solve the VRP with time windows."""

def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                       data['num_vehicles'], data['depot'])

    #used for tracing/debugging
    # routing_parameters = pywrapcp.DefaultRoutingModelParameters()
    # routing_parameters.solver_parameters.trace_propagation = True
    # routing_parameters.solver_parameters.trace_search = True

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager) #, routing_parameters)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Adds Capacity Constraints
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        10000000,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Allow to drop nodes.
    penalty = data['drop_penalty']
    for node in range(1, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        data['maximum_wait_at_location'],  # allow waiting time
        data['maximum_wait_at_location'],  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)


    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
    
    
    # Add driver break constraints
    node_visit_transit = {}
    for n in range(routing.Size()):
        if n >= len(data['time_windows']):
            node_visit_transit[n] = 0
        else:
            node_visit_transit[n] = int(data['demands'][n] * data['service_time'])

    break_intervals = {}
    for v in range(data['num_vehicles']):
        vehicle_break = data['breaks'][v]
        break_intervals[v] = [
            routing.solver().FixedDurationIntervalVar(
                data['driver_break_start'], 
                data['driver_break_end'],
                data['driver_break_duration'],
                False,
                'Break for vehicle {}'.format(v))
        ]
        time_dimension.SetBreakIntervalsOfVehicle(
            break_intervals[v], v, node_visit_transit)

    

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
       routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.log_search = True
    search_parameters.solution_limit = data['solution_limit']


    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        final=pd.DataFrame(columns=['index'])
        dropped_address = pd.DataFrame(columns=['index','address']) # create a df to hold the dropped address
        print_solution(data, manager, routing, solution,final, dropped_address)
    else:
        print("no solutions found")
        
if __name__ == '__main__':
    main()
     #main(sys.argv[1])

Objective: 574
Breaks:
Break for vehicle 0: Start(240) Duration(30)
Break for vehicle 1: Start(240) Duration(30)

  index address           start_time  \
0     0   Add_0  2022-03-07 07:02:00   
1    12  Add_20  2022-03-07 07:02:00   

                                            end_time  
0                                2022-03-07 07:02:00  
1  0   2022-03-07 07:14:00
1   2022-03-07 07:14:0...  
Route for vehicle 0:
0 Time(17,17) -> 12 Time(22,34) -> 8 Time(47,59) -> 16 Time(83,95) -> 15 Time(107,119) -> 4 Time(136,148) -> 6 Time(165,177) -> 3 Time(189,201) -> 10 Time(214,226) -> 14 Time(270,282) -> 5 Time(296,308) -> 18 Time(321,333) -> 2 Time(349,361) -> 13 Time(374,386) -> 11 Time(401,413) -> 9 Time(428,440) -> 0 Time(470,470)
Time of the route: 470min

         index     address           start_time             end_time
0            0       Add_0  2022-03-07 07:02:00  2022-03-07 07:02:00
1           12      Add_20  2022-03-07 07:07:00  2022-03-07 07:19:00
2            8      Add_1